augmenting down_40 train and test

In [11]:
import numpy as np
np.random.seed(62)

In [23]:
import os
import cv2
import numpy as np
import albumentations as A
import random
from tqdm import tqdm

# Global list to record error messages.
error_log = []

# --- Helper functions ---

def get_image_files(folder):
    """
    Returns full paths for all files in folder with common image extensions.
    """
    exts = ['.jpg', '.jpeg', '.png', '.bmp']
    files = []
    for f in os.listdir(folder):
        if os.path.splitext(f)[1].lower() in exts:
            files.append(os.path.join(folder, f))
    return files

def load_annotations(txt_path):
    """
    Reads a YOLO-format annotation file.
    Each line is expected to be:
       class_id x_center y_center width height   (all normalized)
    Returns list of bounding boxes and corresponding labels.
    """
    bboxes = []
    labels = []
    if os.path.exists(txt_path):
        with open(txt_path, 'r') as f:
            for line in f:
                parts = line.strip().split()
                if len(parts) == 5:
                    cls = int(parts[0])
                    bbox = [float(x) for x in parts[1:]]
                    bboxes.append(bbox)
                    labels.append(cls)
    return bboxes, labels

def save_annotations(txt_path, bboxes, labels):
    """
    Saves annotations in YOLO format.
    """
    with open(txt_path, 'w') as f:
        for cls, bbox in zip(labels, bboxes):
            f.write(f"{cls} {bbox[0]:.6f} {bbox[1]:.6f} {bbox[2]:.6f} {bbox[3]:.6f}\n")

def clip_bbox_yolo(bbox):
    """
    Clips a YOLO bbox so that the coordinates remain within [0, 1].
    bbox is [x_center, y_center, width, height].
    """
    xc, yc, w, h = bbox
    xmin = xc - w / 2.0
    xmax = xc + w / 2.0
    ymin = yc - h / 2.0
    ymax = yc + h / 2.0

    xmin = np.clip(xmin, 0.0, 1.0)
    xmax = np.clip(xmax, 0.0, 1.0)
    ymin = np.clip(ymin, 0.0, 1.0)
    ymax = np.clip(ymax, 0.0, 1.0)

    new_xc = (xmin + xmax) / 2.0
    new_yc = (ymin + ymax) / 2.0
    new_w = xmax - xmin
    new_h = ymax - ymin
    return [new_xc, new_yc, new_w, new_h]

def augment_image(image, bboxes, labels, transform, identifier=None):
    """
    Applies an Albumentations transform (with bbox support) to an image and its boxes.
    If an error occurs, logs the error along with image identifier, shape, and original bboxes.
    Then it falls back to returning the original image with manually clipped boxes.
    """
    try:
        augmented = transform(image=image, bboxes=bboxes, labels=labels)
        # Clip bounding boxes in case some values go out-of-bound.
        aug_bboxes = [clip_bbox_yolo(bbox) for bbox in augmented['bboxes']]
        return augmented['image'], aug_bboxes, augmented['labels']
    except Exception as e:
        err_info = (f"Error during augmentation for {identifier if identifier else 'unknown image'}: "
                    f"Image shape: {image.shape}, Original bounding boxes: {bboxes}, "
                    f"Error message: {e}")
        error_log.append(err_info)
        print("Error during augmentation!")
        print("Image shape:", image.shape)
        print("Original bounding boxes:", bboxes)
        print("Error message:", e)
        # Fallback: return original image with manually clipped bounding boxes.
        fixed_bboxes = [clip_bbox_yolo(bbox) for bbox in bboxes]
        return image, fixed_bboxes, labels

def get_transform():
    """
    Returns an Albumentations Compose object that applies:
      - An affine transformation (rotation, translation, scale, shear),
      - A perspective transformation,
      - A horizontal flip.
    The parameters are chosen as reasonable defaults.
    """
    transform = A.Compose([
        A.Affine(
            rotate=(-10, 10),             # Rotate between -10° and 10°
            translate_percent=(-0.1, 0.1),  # Translate up to ±10%
            scale=(0.9, 1.1),               # Scale between 90% and 110%
            shear=(-10, 10),              # Shear up to ±10°
            p=0.7
        ),
        A.Perspective(
            scale=(0.05, 0.1),            # Perspective distortion factor
            p=0.5
        ),
        A.HorizontalFlip(p=0.5)
    ],
    bbox_params=A.BboxParams(format='yolo', label_fields=['labels']))
    return transform

def mosaic_augmentation(image_files, input_dim, mosaic_size_factor=2):
    """
    Creates a mosaic image by randomly combining 4 images.
    Assumes all images are of the same size.
    Returns:
      mosaic_img: the mosaic image.
      mosaic_bboxes: list of adjusted YOLO-format bounding boxes.
      mosaic_labels: list of corresponding labels.
    """
    h, w = input_dim
    mosaic_h, mosaic_w = h * mosaic_size_factor, w * mosaic_size_factor
    mosaic_img = np.full((mosaic_h, mosaic_w, 3), 114, dtype=np.uint8)
    mosaic_bboxes = []
    mosaic_labels = []

    # Choose a random mosaic center.
    cx = random.randint(w // 2, int(w * 1.5))
    cy = random.randint(h // 2, int(h * 1.5))

    # Randomly select 4 images (duplicates allowed)
    indices = [random.randint(0, len(image_files) - 1) for _ in range(4)]

    for i, idx in enumerate(indices):
        img_path = image_files[idx]
        base = os.path.splitext(img_path)[0]
        txt_path = base + '.txt'
        img = cv2.imread(img_path)
        if img is None:
            continue
        ih, iw = img.shape[:2]
        bboxes, labels = load_annotations(txt_path)

        # Determine placement on the mosaic canvas.
        if i == 0:  # top-left quadrant
            x1a, y1a, x2a, y2a = max(cx - iw, 0), max(cy - ih, 0), cx, cy
            x1b, y1b, x2b, y2b = iw - (x2a - x1a), ih - (y2a - y1a), iw, ih
        elif i == 1:  # top-right quadrant
            x1a, y1a, x2a, y2a = cx, max(cy - ih, 0), min(cx + iw, mosaic_w), cy
            x1b, y1b, x2b, y2b = 0, ih - (y2a - y1a), min(iw, x2a - cx), ih
        elif i == 2:  # bottom-left quadrant
            x1a, y1a, x2a, y2a = max(cx - iw, 0), cy, cx, min(cy + ih, mosaic_h)
            x1b, y1b, x2b, y2b = iw - (x2a - x1a), 0, iw, min(y2a - cy, ih)
        elif i == 3:  # bottom-right quadrant
            x1a, y1a, x2a, y2a = cx, cy, min(cx + iw, mosaic_w), min(cy + ih, mosaic_h)
            x1b, y1b, x2b, y2b = 0, 0, x2a - cx, y2a - cy

        # Paste the cropped part of the image onto the mosaic canvas.
        mosaic_img[y1a:y2a, x1a:x2a] = img[y1b:y2b, x1b:x2b]
        pad_x = x1a - x1b
        pad_y = y1a - y1b

        # Adjust and clip bounding boxes for the pasted image.
        for bbox, label in zip(bboxes, labels):
            bx = bbox[0] * iw
            by = bbox[1] * ih
            bw = bbox[2] * iw
            bh = bbox[3] * ih
            x_min = bx - bw / 2
            y_min = by - bh / 2
            x_max = bx + bw / 2
            y_max = by + bh / 2
            x_min_new = x_min + pad_x
            y_min_new = y_min + pad_y
            x_max_new = x_max + pad_x
            y_max_new = y_max + pad_y
            # Clip to the pasted region.
            x_min_new = np.clip(x_min_new, x1a, x2a)
            y_min_new = np.clip(y_min_new, y1a, y2a)
            x_max_new = np.clip(x_max_new, x1a, x2a)
            y_max_new = np.clip(y_max_new, y1a, y2a)
            new_w = x_max_new - x_min_new
            new_h = y_max_new - y_min_new
            if new_w > 1 and new_h > 1:
                new_xc = (x_min_new + x_max_new) / 2 / mosaic_w
                new_yc = (y_min_new + y_max_new) / 2 / mosaic_h
                new_w_norm = new_w / mosaic_w
                new_h_norm = new_h / mosaic_h
                mosaic_bboxes.append([new_xc, new_yc, new_w_norm, new_h_norm])
                mosaic_labels.append(label)
    return mosaic_img, mosaic_bboxes, mosaic_labels

# --- Main processing function ---

def process_dataset(input_folder, output_folder, augment=True, apply_mosaic=True):
    """
    Processes all images in input_folder:
      - Reads the image and its corresponding YOLO annotation.
      - Applies augmentations (affine, perspective, flip) and mosaic (if enabled).
      - Saves augmented images and adjusted annotations in output_folder.
    """
    os.makedirs(output_folder, exist_ok=True)
    image_files = get_image_files(input_folder)
    transform = get_transform()
    
    for img_path in tqdm(image_files, desc="Processing images"):
        base_name = os.path.basename(img_path)
        name_no_ext, ext = os.path.splitext(base_name)
        txt_path = os.path.join(input_folder, name_no_ext + '.txt')
        
        image = cv2.imread(img_path)
        if image is None:
            err_info = f"Warning: Could not read image {img_path}"
            error_log.append(err_info)
            print(err_info)
            continue
        
        ih, iw = image.shape[:2]
        bboxes, labels = load_annotations(txt_path)
        
        if augment:
            # 1. Apply affine/perspective/hflip augmentation.
            aug_img, aug_bboxes, aug_labels = augment_image(image, bboxes, labels, transform, identifier=base_name)
            out_img_path = os.path.join(output_folder, name_no_ext + '_aug' + ext)
            out_txt_path = os.path.join(output_folder, name_no_ext + '_aug.txt')
            cv2.imwrite(out_img_path, aug_img)
            save_annotations(out_txt_path, aug_bboxes, aug_labels)
            
            # 2. Apply mosaic augmentation (if enabled, typically for training only).
            if apply_mosaic:
                try:
                    mosaic_img, mosaic_bboxes, mosaic_labels = mosaic_augmentation(image_files, (ih, iw))
                    mosaic_out_img_path = os.path.join(output_folder, name_no_ext + '_mosaic' + ext)
                    mosaic_out_txt_path = os.path.join(output_folder, name_no_ext + '_mosaic.txt')
                    cv2.imwrite(mosaic_out_img_path, mosaic_img)
                    save_annotations(mosaic_out_txt_path, mosaic_bboxes, mosaic_labels)
                except Exception as e:
                    err_info = (f"Error during mosaic augmentation for {img_path}: "
                                f"Error message: {e}")
                    error_log.append(err_info)
                    print(err_info)
        else:
            # No augmentation: simply copy the original image and annotation.
            out_img_path = os.path.join(output_folder, base_name)
            out_txt_path = os.path.join(output_folder, name_no_ext + '.txt')
            cv2.imwrite(out_img_path, image)
            save_annotations(out_txt_path, bboxes, labels)

# --- Run the augmentation on your datasets ---

if __name__ == "__main__":
    # Define dataset folders.
    train_input = 'data/down_40/train_CLAHE'
    train_output = 'data/down_40/train_CLAHE_augmented'
    val_input = 'data/down_40/val'
    val_output = 'data/down_40/val_augmented'
    
    print("Processing training dataset...")
    process_dataset(train_input, train_output, augment=True, apply_mosaic=True)
    
    print("Processing validation dataset...")
    process_dataset(val_input, val_output, augment=True, apply_mosaic=False)
    
    # Save all recorded error messages into error-log.txt.
    log_file = "error-log.txt"
    with open(log_file, "w") as f:
        for err in error_log:
            f.write(err + "\n")
    print(f"Error log saved to {log_file}")


Processing training dataset...


Processing images:   3%|▎         | 186/5886 [00:02<01:13, 77.37it/s]

Error during augmentation!
Image shape: (367, 256, 3)
Original bounding boxes: [[0.63466, 0.718597, 0.772834, 0.174551], [0.371194, 0.657423, 0.11007, 0.101142], [0.772834, 0.682708, 0.098361, 0.099511], [0.940281, 0.69739, 0.114754, 0.073409]]
Error message: Expected x_max for bbox [0.248243  0.6313215 1.021077  0.8058725 1.       ] to be in the range [0.0, 1.0], got 1.021077036857605.


Processing images:   6%|▋         | 378/5886 [00:04<01:00, 90.98it/s]

Error during augmentation!
Image shape: (274, 300, 3)
Original bounding boxes: [[0.592, 0.228665, 0.82, 0.435449], [0.184, 0.700219, 0.236, 0.582057]]
Error message: Expected x_max for bbox [0.18200001 0.01094049 1.002      0.4463895  2.        ] to be in the range [0.0, 1.0], got 1.0019999742507935.


...

Error during augmentation!
Image shape: (344, 377, 3)
Original bounding boxes: [[0.449126, 0.635889, 0.488076, 0.442509], [0.171701, 0.616725, 0.09539, 0.111498], [0.58744, 0.369338, 0.18601, 0.184669], [0.740064, 0.28223, 0.163752, 0.1777], [0.906995, 0.1777, 0.189189, 0.229965]]
Error message: Expected x_max for bbox [0.8124005 0.0627175 1.0015895 0.2926825 4.       ] to be in the range [0.0, 1.0], got 1.0015895366668701.


Processing images:  99%|█████████▊| 1456/1477 [00:07<00:00, 184.73it/s]

Error during augmentation!
Image shape: (288, 384, 3)
Original bounding boxes: [[0.510156, 0.55625, 0.673438, 0.566667], [0.832031, 0.263542, 0.339062, 0.264583], [0.280469, 0.785417, 0.551562, 0.4125], [0.619531, 0.390625, 0.414062, 0.327083]]
Error message: Expected x_max for bbox [0.6625    0.1312505 1.001562  0.3958335 0.       ] to be in the range [0.0, 1.0], got 1.0015619993209839.
Error during augmentation!
Image shape: (256, 384, 3)
Original bounding boxes: [[0.360156, 0.510539, 0.170313, 0.196721], [0.85, 0.605386, 0.290625, 0.41452], [0.7125, 0.814988, 0.33125, 0.337237], [0.153906, 0.832553, 0.314063, 0.288056], [0.191406, 0.601874, 0.217188, 0.271663], [0.532813, 0.596019, 0.153125, 0.222482]]
Error message: Expected x_min for bbox [-3.1255037e-03  6.8852502e-01  3.1093752e-01  9.7658104e-01
  6.0000000e+00] to be in the range [0.0, 1.0], got -0.003125503659248352.


Processing images: 100%|██████████| 1477/1477 [00:07<00:00, 200.28it/s]

Error log saved to error-log.txt


In [2]:
pip install opency-python

Note: you may need to restart the kernel to use updated packages.


ERROR: Could not find a version that satisfies the requirement opency-python (from versions: none)
ERROR: No matching distribution found for opency-python

[notice] A new release of pip is available: 23.2.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


# testing bounding box

In [2]:
import cv2
import numpy as np
import matplotlib.pyplot as plt


image_path = "data/down_40/train_CLAHE_augmented/2015_00004_mosaic.jpg"
annotation_path = "data/down_40/train_CLAHE_augmented/2015_00004_mosaic.txt"
image = cv2.imread(image_path)
if image is None:
    raise FileNotFoundError(f"Error: Image not found at {image_path}")


img_h, img_w, _ = image.shape
bboxes = []
with open(annotation_path, "r") as f:
    for line in f.readlines():
        parts = line.strip().split()
        class_id = int(float(parts[0]))  
        x_center, y_center, width, height = map(float, parts[1:])  
        x1 = int((x_center - width / 2) * img_w)
        y1 = int((y_center - height / 2) * img_h)
        x2 = int((x_center + width / 2) * img_w)
        y2 = int((y_center + height / 2) * img_h)

        bboxes.append((x1, y1, x2, y2, class_id))

# bbox
for (x1, y1, x2, y2, class_id) in bboxes:
    color = (0, 255, 0)  # grin
    cv2.rectangle(image, (x1, y1), (x2, y2), color, 2)
    cv2.putText(image, str(class_id), (x1, y1 - 5), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)

image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
plt.figure(figsize=(8, 6))
plt.imshow(image_rgb)
plt.axis("off")  
plt.show()


ModuleNotFoundError: No module named 'cv2'